## Grabbing the Data
-----

### Setup:
---

In [ ]:
# Import dependencies
from pathlib import Path
import pandas as pd

### Grab NFL Game Data from Past Years:
---

In [ ]:
# Try With a Single Year (2022) First


### Export Past Game Data as CSV Files:
---

In [ ]:
# Export CSV's for Each Team to ../03-Wrangling_Data/Grabbing_Data_Exports
